<h3>VVI</h3>

In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,SerpAPIWrapper
from langchain_core.tools import Tool

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [50]:
load_dotenv(find_dotenv("../.env"))

True

In [51]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<h3>Tavily Search Tool</h3>

In [52]:
searchTavily=TavilySearchResults(max_results=4)

In [53]:
searchTavily.invoke(input="How is the weather in Kolkata?")

[{'title': 'Kolkata weather update: Heavy rain expected - Times of India',
  'url': 'https://timesofindia.indiatimes.com/city/kolkata/kolkata-weather-update-expect-moderate-rain-showers-and-high-humidity-today/articleshow/122101335.cms',
  'content': 'Kolkata residents are set to experience moderate rainfall on June 27, 2025, with temperatures ranging between 33.2°C and 27.3°C amid high humidity of 76%. The city anticipates 14.76 mm of precipitation with a 91% chance of rain, accompanied by moderate winds reaching up to 20.9 km/h.The average temperature will remain around 29.3°C, offering some relief from typical summer conditions. The combination of rain and elevated humidity levels may create uncomfortable conditions throughout the day. [...] Karnataka Beauty Parlour Raid\n\nUttarakhand Bus Accident\n\nNoida Fire\n\nF35 Fighter Jet\n\nJagannath Rath Yatra\n\nJaggu Bhagwanpuria\n\nKarnataka Beauty Parlour Raid\n\nUttarakhand Bus Accident\n\nKolkata weather update: Heavy rain expected\

<h3>Wikipedia Search</h3>

In [54]:
wikipediaSearch=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

<h3>SerpAPI Search</h3>

In [55]:
serpAPISearch=Tool(
    func=SerpAPIWrapper(search_engine="bing").run,
    description="Search Tool from SerpAPI",
    name="serpAPI_Search")

<h3>Retrievers</h3>

In [56]:
# Loaders
loader=WebBaseLoader(web_path="https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms")

In [57]:
docs=loader.load()

In [58]:
docs

[Document(metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content='Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdisha

In [59]:
# Split Documents
documentsObj=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [60]:
documents=documentsObj.split_documents(documents=docs)

In [61]:
documents

[Document(metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content="Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdisha

In [62]:
len(documents)

18

In [63]:
# Use Vector DB to Insert Documents
vector=FAISS.from_documents(documents=documents,embedding=OpenAIEmbeddings())

In [64]:
retriever=vector.as_retriever()

In [32]:
retriever.invoke(input="Who is the prime accussed?")

[Document(id='1039e4e9-ad83-4915-badc-f618f9d81916', metadata={'source': 'https://timesofindia.indiatimes.com/india/kolkata-shocker-victim-alleges-rape-was-videotaped-what-initial-probe-revealed/articleshow/122113479.cms', 'title': 'Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of India', 'description': 'India News: Kolkata Police reported the arrest of Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader, for allegedly raping a law student within c', 'language': 'en'}, page_content='filming the assault and threatening to release it.  NEW DELHI: Kolkata Police on Thursday said that the prime accused Manojit Mishra in the rape of an law student inside college premises was former president of the college’s Trinamool Congress Chhatra Parishad unit and the organisational secretary of the south Kolkata wing of the TMC’s student body.Photographs of the key accused available in the public domain also showed him in close pr

<h3>Create Retriever Tool</h3>

In [72]:
retrieverTool=create_retriever_tool(
    retriever=retriever,
    name="Kolkata_harrassment_accussed",
    description="Search for information about the Kolkata Harrassment incident. For any questions about the Kolkata Harrassment, you must use this tool!"
)

In [73]:
# Use Both the Tools - search and retrievertool to use in the downstream
tools=[
    retrieverTool,
    searchTavily,
    serpAPISearch,
    wikipediaSearch]

In [74]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-functions-agent")

In [75]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [76]:
# The agent is responsible for taking in input and deciding what actions to take
# The agent does not execute those actions, that is done by the agent executor

agent=create_tool_calling_agent(llm=llmOpenAI,tools=tools,prompt=prompt)

In [77]:
agentExecutor=AgentExecutor(tools=tools,agent=agent,verbose=True)

In [78]:
agentExecutor.invoke(input={"input":"hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [79]:
agentExecutor.invoke(input={"input":"Who is the Prime Accussed?"})



> Entering new AgentExecutor chain...

Invoking: `Kolkata_harrassment_accussed` with `{'query': 'Prime Accused'}`


filming the assault and threatening to release it.  NEW DELHI: Kolkata Police on Thursday said that the prime accused Manojit Mishra in the rape of an law student inside college premises was former president of the college’s Trinamool Congress Chhatra Parishad unit and the organisational secretary of the south Kolkata wing of the TMC’s student body.Photographs of the key accused available in the public domain also showed him in close proximity to several leaders of the state’s ruling dispensation.Law Student Gangraped Inside Kolkata College; TMC Youth Leader Among 3 Arrested | Shocking DetailsAccording to college authorities, the main accused was also engaged as a temporary non-teaching staff at the institution for a renewable contractual period of 45 days. He was also a practising criminal lawyer at the Alipore Police and Sessions Court.Meanwhile, the vice principal of

{'input': 'Who is the Prime Accussed?',
 'output': 'The prime accused in the Kolkata harassment incident is Manojit Mishra. He is a former president of the college’s Trinamool Congress Chhatra Parishad unit and the organizational secretary of the south Kolkata wing of the TMC’s student body. Manojit Mishra is accused of raping a law student inside college premises and filming the assault, threatening to release the footage.'}

In [80]:
agentExecutor.invoke(input={'input':"What is the weather at Kolkata now?"})



> Entering new AgentExecutor chain...

Invoking: `serpAPI_Search` with `weather in Kolkata`


{'type': 'weather_result', 'temperature': '82', 'unit': 'Fahrenheit', 'precipitation': '75%', 'humidity': '90%', 'wind': '5 mph', 'location': 'Kolkata, West Bengal, India', 'date': 'Saturday 1:00 AM', 'weather': 'Light rain'}The current weather in Kolkata is as follows:
- Temperature: 82°F
- Precipitation: 75%
- Humidity: 90%
- Wind: 5 mph
- Weather: Light rain

Please let me know if you need more information.

> Finished chain.


{'input': 'What is the weather at Kolkata now?',
 'output': 'The current weather in Kolkata is as follows:\n- Temperature: 82°F\n- Precipitation: 75%\n- Humidity: 90%\n- Wind: 5 mph\n- Weather: Light rain\n\nPlease let me know if you need more information.'}

<h3> Adding in Memory</h3>

In [81]:
messageHistory=ChatMessageHistory()

In [82]:
agentWithChatHistory=RunnableWithMessageHistory(
    runnable=agentExecutor,
    get_session_history=lambda session_id: messageHistory,
    input_message_key="input",
    history_messages_key="chat_history"
)

In [83]:
agentWithChatHistory.invoke(
    input={"input":"hi! My Name is Ritish"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Hello Ritish! How can I assist you today?

> Finished chain.


{'input': 'hi! My Name is Ritish',
 'chat_history': [],
 'output': 'Hello Ritish! How can I assist you today?'}

In [84]:
agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Your name is Ritish. How can I assist you today, Ritish?

> Finished chain.


{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ritish. How can I assist you today, Ritish?'}

In [85]:
agentWithChatHistory.invoke(
    input={"input":"What is the name of the victim?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Kolkata Harrassment incident'}`


No good Wikipedia Search Result was found
Invoking: `Kolkata_harrassment_accussed` with `{'query': 'victim'}`


Kolkata shocker: Victim alleges rape was videotaped; what initial probe revealed | India News - Times of IndiaEditionININUSEnglishEnglishहिन्दीमराठीಕನ್ನಡதமிழ்বাংলাമലയാളംతెలుగుગુજરાતીSign InTOIindiaOperation SindoorCancel PatriarchyMaharashtraDelhiKarnatakaTamil NaduTelanganaUttar PradeshWest BengalGujaratMadhya PradeshBiharChandigarhRajasthanArunachal PradeshAndhra PradeshAssamChhattisgarhGoaHaryanaHimachal PradeshJammu KashmirJharkhandKeralaManipurMeghalayaMizoramNagalandOdishaPunjabSikkimTripuraUttarakhandAndaman Nicobar IslandsDadra Nagar HaveliDaman DiuLakshadweepPondicherryWeb StoriesSaving Our StripesDigital ArrestToday's ePaperNewsIndia NewsKolkata shocker: Victim alleges rape was videotaped; what initial probe revealedTrendingBest education system Countrie

{'input': 'What is the name of the victim?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={})],
 'output': 'The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.'}

In [86]:
agentWithChatHistory.invoke(
    input={"input":"Who won in the Iran-Israel war?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Iran-Israel War'}`


Page: Iran–Israel war
Summary: An armed conflict between Iran and Israel began on 13 June 2025, when Israel launched surprise attacks on key military and nuclear facilities in Iran. In the opening hours of the war, Israeli air and ground forces assassinated some of Iran's prominent military leaders, nuclear scientists, and politicians; and damaged or destroyed Iran's air defenses and some of its nuclear and military facilities. Iran retaliated by launching missiles at cities and military sites in Israel. The Iran-allied Houthis in Yemen also fired several missiles at Israel, in an adjunct of the Red Sea crisis. The United States, which defended Israel against Iranian missiles and drones, took offensive action on the ninth day of the war by striking three Iranian nuclear sites. The Houthis took the American strikes as a "declaration of war" and unilaterally ended their ceasefire with the

{'input': 'Who won in the Iran-Israel war?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.', additional_kwargs={}, response_metadata={})],
 'output': 'The information provided does not specify a clear winner in the Iran-Israel war. 

In [87]:
agentWithChatHistory.invoke(
    input={"input":"Which party won the election in Jammu and Kashmir?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Jammu and Kashmir election results winner'}`


[{'title': 'J&K assembly election results: Check full list of winners', 'url': 'https://www.hindustantimes.com/india-news/jk-full-winners-list-jammu-kashmir-assembly-election-2024-today-oct-8-omar-abdullah-iltija-mufti-ravinder-raina-101728378222213.html', 'content': 'J&K election results full list of winners: After trends showed that the Congress-National Conference pre-poll alliance had crossed the majority mark in Jammu and Kashmir, jubilant Jammu and Kashmir National Conference (JKNC) workers celebrated in Srinagar on Tuesday.\n\nJKNC leader Omar Abdullah won the Budgam assembly seat by a margin of over 18,000 votes and also the Ganderbal seat. (Waseem Andrabi/HT photo)") [...] According to the latest ECI figures, JKNC has secured 42 seats, while Congress has won 6 seats. Allied party CPM has won one seat. The BJP has claimed 29 seats. Mehb

{'input': 'Which party won the election in Jammu and Kashmir?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', additional_kwargs

In [88]:
agentWithChatHistory.invoke(
    input={"input":"What's are the questions that I have asked so far?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
So far, you have asked the following questions:

1. What's my name?
2. What is the name of the victim?
3. Who won in the Iran-Israel war?
4. Which party won the election in Jammu and Kashmir?

Is there anything else you would like to know or ask about?

> Finished chain.


{'input': "What's are the questions that I have asked so far?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', additional_kwargs

In [89]:
response=agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
Your name is Ritish. How can I assist you today, Ritish?

> Finished chain.


In [92]:
response

{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ritish! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish. How can I assist you today, Ritish?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the name of the victim?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The victim in the Kolkata Harrassment incident is a law student who alleged that she was raped by the accused, who also filmed the assault and threatened to release it. The prime accused is Manujit Mishra, a former Trinamool Congress Chhatra Parishad leader.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who won in the Iran-Israel war?', additional_kwargs={}, response_metadata={}),
  AIMes